In [1]:
# import packages
import matplotlib.pyplot as plt
import numpy as np

# to print plots inline
%matplotlib inline

# set out parameters
rho = 0.7605
mu = 0.0
sigma_eps = 0.213

# draw our shocks
num_draws = 100000 # number of shocks to draw
eps = np.random.normal(0.0, sigma_eps, size=(num_draws))

# Compute z
z = np.empty(num_draws)
z[0] = 0.0 + eps[0]
for i in range(1, num_draws):
    z[i] = rho * z[i - 1] + (1 - rho) * mu + eps[i]


In [2]:
sigma_z = sigma_eps / ((1 - rho ** 2) ** (1 / 2))
#print('Theoretical sigma_z = ', sigma_z)

# from our simulation:
sigma_z_simul = z.std()
#print('Simulated sigma_z = ', sigma_z_simul)

In [3]:
from scipy.stats import norm

# Compute cut-off values
N = 9  # number of grid points (will have one more cut-off point than this)
z_cutoffs = (sigma_z * norm.ppf(np.arange(N + 1) / N)) + mu
print('Cut-off values = ', z_cutoffs)

Cut-off values =  [       -inf -0.40040229 -0.25084498 -0.14128994 -0.04582867  0.04582867
  0.14128994  0.25084498  0.40040229         inf]


In [4]:
# compute grid points for z
z_grid = ((N * sigma_z * (norm.pdf((z_cutoffs[:-1] - mu) / sigma_z)
                              - norm.pdf((z_cutoffs[1:] - mu) / sigma_z)))
              + mu)
print('Grid points = ', z_grid)

Grid points =  [-0.55913938 -0.32004072 -0.19425291 -0.09290094  0.          0.09290094
  0.19425291  0.32004072  0.55913938]


In [5]:
"""
# import packages
import scipy.integrate as integrate

# define function that we will integrate
def integrand(x, sigma_z, sigma_eps, rho, mu, z_j, z_jp1):
    val = (np.exp((-1 * ((x - mu) ** 2)) / (2 * (sigma_z ** 2)))
            * (norm.cdf((z_jp1 - (mu * (1 - rho)) - (rho * x)) / sigma_eps)
               - norm.cdf((z_j - (mu * (1 - rho)) - (rho * x)) / sigma_eps)))
    
    return val

# compute transition probabilities
pi = np.empty((N, N))
for i in range(N):
    for j in range(N):
        results = integrate.quad(integrand, z_cutoffs[i], z_cutoffs[i + 1],
                                 args = (sigma_z, sigma_eps, rho, mu,
                                         z_cutoffs[j], z_cutoffs[j + 1]))
        pi[i,j] = (N / np.sqrt(2 * np.pi * sigma_z ** 2)) * results[0]
        
# print('Transition matrix = ', pi)
# print('pi sums = ', pi.sum(axis=0), pi.sum(axis=1))
"""

In [26]:
#By the theory of Adda-Cooper methods, each value should have equal probability
probs = 1./(len(z_grid))*np.ones(len(z_grid))

[ 0.11111111  0.11111111  0.11111111  0.11111111  0.11111111  0.11111111
  0.11111111  0.11111111  0.11111111]


In [8]:
"""
# Simulate the Markov process - will make this a function so can call later
def sim_markov(z_grid, pi, num_draws):
    # draw some random numbers on [0, 1]
    u = np.random.uniform(size=num_draws)

    # Do simulations
    z_discrete = np.empty(num_draws)  # this will be a vector of values 
    # we land on in the discretized grid for z
    N = z_grid.shape[0]
    oldind = int(np.ceil((N - 1) / 2)) # set initial value to median of grid
    z_discrete[0] = z_grid[oldind]  
    for i in range(1, num_draws):
        sum_p = 0
        ind = 0
        while sum_p < u[i]:
            sum_p = sum_p + pi[ind, oldind]
#             print('inds =  ', ind, oldind)
            ind += 1
        if ind > 0:
            ind -= 1
        z_discrete[i] = z_grid[ind]
        oldind = ind
                            
    return z_discrete


# Call simulation function to get simulated values
z_discrete = sim_markov(z_grid, np.transpose(pi), num_draws)


#We now have z values stored as z-grid and the 
#probability of each stored as z-discrete
"""

In [24]:
"""
from scipy.sparse import linalg as sla
eval, evec = sla.eigs(pi.T, k=1, which='LM')
evec=(evec/np.sum(evec)).real
"""

[ 1.+0.j]
[[-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]
 [-0.33333333+0.j]]
[[ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]
 [ 0.11111111]]


In [6]:
# specify parameters
alpha_k = 0.297
alpha_l = 0.65
delta = 0.154
psi = 1.08
w = 0.7
r= 0.04
z = 1
betafirm = (1 / (1 + r))

In [ ]:
dens = 3
# put in bounds here for the capital stock space
kstar = ((((1 / betafirm - 1 + delta) * ((w / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (z ** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
kvec = K[::-1]
sizek = kvec.shape[0]

In [ ]:
# operating profits, op
op = np.mean(((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l))) *
      ((kvec ** alpha_k) ** (1 / (1 - alpha_l)))))

# firm cash flow, e
e = np.zeros((sizek, sizek))
for i in range(sizek):
    for j in range(sizek):
            e[i, j] = (op[i] - kvec[j] + ((1 - delta) * kvec[i]) -
                       ((psi / 2) * ((kvec[j] - ((1 - delta) * kvec[i])) ** 2)
                        / kvec[i]))